In [115]:
import os
import requests
import random
import pickle
KEY = '91mTqXFEeSaCIbjC0fuaB1RABk4HANqS4I91qRBN'
KEY2 = 'flLQZGa0kx8opciUA6M7R9iLPwfFPj5yFKKlCiVb'
MOST_RECENT_HR_BILL = 1490
MOST_RECENT_S_BILL = 606
CONGRESS = 115

In [113]:
def getJSON(webpage):
    headers = {
        'X-API-Key': KEY
    }
    r = requests.get(webpage, headers=headers)
    return r.json()

def getSponsors(bill_id, congress):
    '''Returns (PrimarySponsor, [cosponsors]) for specified bill_id and congress strings. Returns () if faulty'''
    edge = getJSON('https://api.propublica.org/congress/v1/' + str(congress) + '/bills/' + bill_id + '/cosponsors.json')
    if edge['status'] == 'OK':
        return (edge['results'][0]['sponsor_id'], [cosponsor['cosponsor_id'] for cosponsor in edge['results'][0]['cosponsors']])
    else:
        return ()
    
def getMembers(congress):
    '''takes congress session and Returns dictioanry of congress member ids with value 0'''
    senate_members = getJSON('https://api.propublica.org/congress/v1/'+ str(congress) +'/senate/members.json')
    house_members = getJSON('https://api.propublica.org/congress/v1/'+ str(congress) +'/house/members.json')
    senateList, houseList = [],[]
    if senate_members['status'] == 'OK':
        senateList = [(senator['id'],0) for senator in senate_members['results'][0]['members']]
        print('senate members', len(senateList))
    if house_members['status'] == 'OK':
        houseList = [(rep['id'],0) for rep in house_members['results'][0]['members']]
        print('house members', len(houseList))
    return dict(senateList + houseList)


    
    
def buildGraph(members, congress):
    graph = {}
    for m in members:
        graph[m] = []
    
    
    for i in range(1, MOST_RECENT_HR_BILL+1):
        print("At HR{0}".format(i))
        edge = getSponsors("hr{0}".format(i), congress)
        if edge: # not (,)
            graph[edge[0]].extend(edge[1])
    
    for i in range(1, MOST_RECENT_S_BILL+1):
        print("At S{0}".format(i))
        edge = getSponsors("s{0}".format(i), congress)
        if edge: # not (,)
            graph[edge[0]].extend(edge[1])
        
    return graph


def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)
    
    
    

In [91]:
members = getMembers(CONGRESS)

senate members 101
house members 441


In [109]:
graph = buildGraph(members, CONGRESS)
print(graph)

At HR1
At HR2
At HR3
At HR4
At HR5
At HR6
At HR7
At HR8
At HR9
At HR10
At HR11
At HR12
At HR13
At HR14
At HR15
At HR16
At HR17
At HR18
At HR19
At HR20
At HR21
At HR22
At HR23
At HR24
At HR25
At HR26
At HR27
At HR28
At HR29
At HR30
At HR31
At HR32
At HR33
At HR34
At HR35
At HR36
At HR37
At HR38
At HR39
At HR40
At HR41
At HR42
At HR43
At HR44
At HR45
At HR46
At HR47
At HR48
At HR49
At HR50
At HR51
At HR52
At HR53
At HR54
At HR55
At HR56
At HR57
At HR58
At HR59
At HR60
At HR61
At HR62
At HR63
At HR64
At HR65
At HR66
At HR67
At HR68
At HR69
At HR70
At HR71
At HR72
At HR73
At HR74
At HR75
At HR76
At HR77
At HR78
At HR79
At HR80
At HR81
At HR82
At HR83
At HR84
At HR85
At HR86
At HR87
At HR88
At HR89
At HR90
At HR91
At HR92
At HR93
At HR94
At HR95
At HR96
At HR97
At HR98
At HR99
At HR100
At HR101
At HR102
At HR103
At HR104
At HR105
At HR106
At HR107
At HR108
At HR109
At HR110
At HR111
At HR112
At HR113
At HR114
At HR115
At HR116
At HR117
At HR118
At HR119
At HR120
At HR121
At HR122
At HR123
A

In [116]:
save_obj(graph, 'graph')

In [117]:
load_dict = load_obj('graph')
print(load_dict)

{'H001053': [], 'M001170': ['B000575', 'B000575', 'B000944', 'P000449', 'C001075', 'C001095', 'B001236'], 'S001190': ['Y000065'], 'C001103': ['A000372', 'B000490', 'C001093', 'F000465', 'G000560', 'H001071', 'J000288', 'L000287', 'L000583', 'S001189', 'S001157', 'W000810', 'W000814', 'B001274', 'M001177', 'G000565', 'H001052', 'P000601', 'R000409', 'H001071', 'B001293', 'R000603', 'G000552', 'B001248', 'O000168', 'D000616', 'K000362', 'B001291', 'S001189', 'Y000065', 'S000583'], 'D000614': ['W000812', 'M001190', 'M001187', 'S001192', 'P000606', 'G000576', 'W000814', 'L000566', 'S000522', 'J000255', 'C001103', 'F000448', 'M001195', 'M001198', 'F000460', 'O000168', 'B001299', 'R000604', 'L000569', 'A000372', 'R000591', 'C001078', 'G000552', 'N000147', 'M001187', 'S001191', 'C001053', 'C001059', 'G000576', 'F000460', 'D000620', 'P000593'], 'V000130': ['C001097', 'S000248', 'G000551', 'P000608', 'D000598', 'L000580', 'S001200', 'V000132', 'G000581', 'G000574', 'H001048'], 'C001078': ['C000

In [137]:
def ranker(members, graph, p=0.15, iterations=100000):
    assert len(graph) == len(members)
    member_list = list(graph.keys())
    current = random.choice(member_list)
    members[current] += 1
    for i in range(iterations):
        if i % 10 == 0:
            print("Iteration: {0}".format(i))
        if len(graph[current]) == 0 or random.random() < p:
            current = random.choice(member_list)
            print("reset")
        else:
            neighbors = graph[current]
            current = neighbors[random.randint(0, len(neighbors)-1)]
            print("Continue")
        members[current] += 1

In [138]:
ranker(members, graph)
member_ranks = [pair[0] for pair in sorted(members.items(), key = lambda item: item[1], reverse=True)]
top_score = members[member_ranks[0]]
for i in range(len(member_ranks)):
    print("%d %f: %s" % (i+1,members[member_ranks[i]]/top_score, member_ranks[i]))

Iteration: 0
reset
Continue
Continue
reset
reset
Continue
Continue
Continue
reset
reset
Iteration: 10
Continue
reset
Continue
Continue
Continue
Continue
reset
Continue
Continue
Continue
Iteration: 20
Continue
Continue
reset
Continue
Continue
Continue
Continue
Continue
Continue
Continue
Iteration: 30
Continue
Continue
Continue
Continue
Continue
reset
Continue
reset
reset
reset
Iteration: 40
Continue
Continue
Continue
Continue
Continue
reset
Continue
Continue
Continue
reset
Iteration: 50
reset
Continue
Continue
reset
Continue
Continue
reset
reset
Continue
Continue
Iteration: 60
Continue
Continue
Continue
reset
reset
Continue
Continue
Continue
Continue
Continue
Iteration: 70
Continue
Continue
Continue
Continue
reset
Continue
reset
Continue
Continue
Continue
Iteration: 80
Continue
Continue
reset
Continue
reset
reset
Continue
reset
Continue
Continue
Iteration: 90
Continue
Continue
reset
Continue
Continue
Continue
Continue
reset
reset
Continue
Iteration: 100
Continue
Continue
Continue
Contin

In [129]:
getJSON('https://api.propublica.org/congress/v1/members/K000382.json')

{'copyright': 'Copyright (c) 2017 Pro Publica Inc. All Rights Reserved.',
 'results': [{'crp_id': '',
   'cspan_id': '62650',
   'current_party': 'D',
   'date_of_birth': '1956-09-05',
   'domain': 'kuster.house.gov',
   'facebook_account': 'CongresswomanAnnieKuster',
   'first_name': 'Ann',
   'gender': 'F',
   'google_entity_id': '/m/064p47_',
   'govtrack_id': '412557',
   'icpsr_id': '21340',
   'in_office': 'true',
   'last_name': 'Kuster',
   'member_id': 'K000382',
   'middle_name': 'McLane',
   'most_recent_vote': '2017-03-10',
   'roles': [{'bills_cosponsored': '0',
     'bills_sponsored': '3',
     'chamber': 'House',
     'committees': [],
     'congress': '115',
     'district': '2',
     'end_date': '2019-01-03',
     'fec_candidate_id': 'H0NH02181',
     'leadership_role': '',
     'missed_votes_pct': '3.16',
     'ocd_id': 'ocd-division/country:us/state:nh/cd:2',
     'office': '',
     'party': 'D',
     'phone': '202-225-5206',
     'seniority': '6',
     'start_date':